# 데이터 로드 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
from glob import glob 
import os 

In [ ]:
!tar -xvf '/content/drive/MyDrive/데이터보관/공유용/train_features.tar'

In [149]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
csvs_dir = glob('/content/katl/*.bz2')
csvs_dir.sort()
csvs_dir = pd.DataFrame(csvs_dir,columns=['dir'])
csvs_name = list(csvs_dir['dir'].apply(lambda x : x.split('katl/')[-1].split('.csv')[0].split('_',maxsplit=1)[-1]))

In [8]:
def make_dir(dir,df):
  csv_dir = df[df['dir'].apply(lambda x : dir in x )].values[0][0]
  return csv_dir 

In [ ]:
for csv_name in tqdm(csvs_name):
  exec(f"{csv_name} = pd.read_csv(make_dir('{csv_name}',csvs_dir))")

In [146]:
airport_config_df = pd.read_csv(make_dir('config',csvs_dir))

# 데이터 전처리 

## 15분 단위 분할 

In [136]:
airport_config_df['timestamp'] = pd.to_datetime(airport_config_df['timestamp'])
airport_config = airport_config_df.sort_values(by='timestamp')

In [137]:
airport_config = pd.DataFrame(airport_config.set_index('timestamp').airport_config.resample('30min').ffill().dropna()).reset_index()

In [138]:
airport_config

,timestamp,airport_config
0,2020-11-01 01:30:00,D_8R_9L_A_10_8L_9R
1,2020-11-01 02:00:00,D_8R_9L_A_10_8L_9R
2,2020-11-01 02:30:00,D_8R_9L_A_10_8L_9R
3,2020-11-01 03:00:00,D_8R_9L_A_10_8L_9R
4,2020-11-01 03:30:00,D_8R_9L_A_10_8L_9R
...,...,...
17512,2021-10-31 21:30:00,D_26L_27R_A_26R_27L_28
17513,2021-10-31 22:00:00,D_26L_27R_A_26R_27L_28
17514,2021-10-31 22:30:00,D_26L_27R_A_26R_27L_28
17515,2021-10-31 23:00:00,D_26L_27R_A_26R_27L_28


## One hot Encoding 

In [82]:
open_train_label = pd.read_csv('/content/drive/MyDrive/데이터보관/공유용/open_train_labels (1).csv.bz2')

In [83]:
labels = pd.get_dummies(open_train_label[open_train_label['airport']=='katl'],columns = ['config']).columns[4:]
labels = list(pd.Series(labels).apply(lambda x : x.split(':')[-1]))

In [139]:
for label in labels:
  airport_config[label]='-'

In [140]:
def one_hot(config,label):
  if config == label:
    return 1 
  else:
    return 0

In [141]:
for label in labels:
  airport_config[label] = airport_config['airport_config'].apply(lambda x: one_hot(x,label))

In [142]:
airport_config

,timestamp,airport_config,D_10_8L_A_10_8L,D_10_8R_9L_A_10_8L_9R,D_10_8R_A_10_8R,D_26L_27L_A_26R_27L_28,D_26L_27R_28_A_26R_27L_28,D_26L_27R_A_26L_27L_28,D_26L_27R_A_26L_27R,D_26L_27R_A_26L_27R_28,...,D_26R_28_A_26R_28,D_8L_9L_A_10_8L_9R,D_8R_9L_A_10_8L_9R,D_8R_9L_A_10_8R_9R,D_8R_9L_A_10_9R,D_8R_9L_A_8L_9R,D_8R_9L_A_8R_9L,D_8R_9R_A_10_8L_9R,D_9L_A_9R,other
0,2020-11-01 01:30:00,D_8R_9L_A_10_8L_9R,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2020-11-01 02:00:00,D_8R_9L_A_10_8L_9R,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2020-11-01 02:30:00,D_8R_9L_A_10_8L_9R,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2020-11-01 03:00:00,D_8R_9L_A_10_8L_9R,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2020-11-01 03:30:00,D_8R_9L_A_10_8L_9R,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17512,2021-10-31 21:30:00,D_26L_27R_A_26R_27L_28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17513,2021-10-31 22:00:00,D_26L_27R_A_26R_27L_28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17514,2021-10-31 22:30:00,D_26L_27R_A_26R_27L_28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17515,2021-10-31 23:00:00,D_26L_27R_A_26R_27L_28,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [185]:
1459*12

17508

# train - valid - test 

In [186]:
input_df = np.array(airport_config.drop(columns = ['timestamp','airport_config']).drop(np.arange(17508,17517)))
input_df = input_df.reshape(-1,12,27)

In [187]:
input_df.shape

(1459, 12, 27)

#모델
**softmax 방식**
- 과거 6시간의 airport configuration 정보를  이용해 미래의 30분 이후 ~ 6시간 이후를 예측 
- ex : 11시에 예측을 시행한다 할 경우 11시 이전의 6시간(5~11시)까지의 데이터를 이용해 11시30분, 12시 ~ 17시 까지의 데이터들을 예측 
- 예측 하는 방식은 one hot encoding 형태의 configuration을 softmax로 확률 예측 
- 분류에선 softmax score로 분류를 했지만 여기서는 softmax score를 확률로써 그대로 사용 

**Regression 방식**
- 과거의 6시간 데이터를 이용해 미래의 6시간을 통으로 예측 
- input : 과거 6시간 데이터 -> output : 미래 6시간  
- regression을 통해 나온 결과들을 min,max scaling을 통해 sum 1로 맞춰줌 

##사전설정

In [148]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import Recall
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Bidirectional, LSTM, BatchNormalization,Input,Dropout,Dense,Conv2D,Concatenate
from tensorflow.keras.utils import plot_model
from keras.layers.pooling import GlobalAveragePooling2D,GlobalAveragePooling3D,AveragePooling3D

##콜백

## 모델

In [173]:
input = Input((12,27))
input = Bidirectional(LSTM(12,return_sequences=True))(input)
input = LSTM(12)(input)
out = Dense(27,activation='softmax')(input)

model = Model(input,out)

In [174]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 12)]              0         
                                                                 
 dense_1 (Dense)             (None, 27)                351       
                                                                 
Total params: 351
Trainable params: 351
Non-trainable params: 0
_________________________________________________________________


## Compile

##Cross Validation

# 예측 및 평가